# Table of Contents
1. [Things to Do](#Things-to-Do)
1. [Introduction](#Introduction)
1. [Set up Python](#Set-up-Python)
2. [Functions](#Functions)
3. [Getting Data](#Get-Data)
4. [Analyze Data](#Analyze-Data):
    - [Population Size](#Population-Size) - noticeable changes
    - [Sex Distribution](#Sex-Distribution) - does not change
    - [Tail Condition Distribution](#Tail-Condition-Distribution)- noticeable change
        - [Severity of Autotomy](#Severity-of-Autotomy)- noticeable change
    - [Location](#Location)
    - [Morphometrics](#Morphometrics):
        - [SVL](#SVL)
        - [TL](#TL)
        - [RTL](#RTL)
        - [Mass](#Mass)- noticeable change (Check Males)
    - [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
    - [Captures](#Captures)
    - [Growth](#Growth)
        - [SVL Growth](#SVL-Growth)
        - [TL Growth](#TL-Growth)
        - [RTL Growth](#RTL-Growth)
        - [Mass Growth](#Mass-Growth)
    - [Correlations to Population](#Correlations-to-Population)
5. [Export Files](#Export-Files)

# Things to Do

Also search for "to be done"

## [Resume Here](#resume)

## Introduction

This notebook contains code and output of descriptive analyses for the 2000-2017 CC dataset after cleaning.

The objectives of this notebook are to describe the community of the _Sceloporus jarrovii_ and _Sceloporus virgatus_ lizards in the Crystal Creek wash from 2000 until 2017.  The population demographic metrics we examine are: [population size](#Population-Size), [sex distribution](#Sex-Distribution), [tail condition distribution](#Tail-Condition-Distribution), [location](#Location), [morphometrics](#Morphometrics) -- [SVL](#SVL), [TL](#TL), [RTL](#RTL), [mass](#Mass) --,  [survival and rates and likelihood of recapture](#Survival-and-Rates-and-Likelihood-of-Recapture), and [growth](#Growth).

We will examine these metrics and interactions among them with particular interest in the impact of environmental factors from year to year.


##  Set up Python

First we will need to set up the python environment, importing the necessary packages and setting the display options.

[Top](#Table-of-Contents)

In [25]:
import pandas as pd
import numpy as np
import pingouin as pg
import os, glob, logging
from summary_functions import *
from scipy import stats
from monthlit import *
from prettyprint import *


import plotly
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
# plotly.tools.set_config_file(world_readable=True)


# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 50

### Setting File Locations

In [26]:
deviceDict = {'dataBig':{'source':'S:/Chris/TailDemography/TailDemography/outputFiles'
                         ,'log':'S:/Chris/TailDemography/TailDemography/Scripts and notes/Descriptive/'
                         ,'output':'S:/Chris/TailDemography/TailDemography/outputFiles/'},
              'silverSurfer':{'source':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles'
                              ,'log':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                              ,'output':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles\\Descriptive\\'}
              ,'dataPers':{'source':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
                           ,'log': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                           ,'output':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles/Descriptve/'}
             ,'gandolf':{'source':'C:/Users/craga/Google Drive/TailDemography/outputFiles'
                           ,'log': 'C:/Users/craga/Google Drive/TailDemography/Scripts and notes/Descriptive/'
                           ,'output':'C:/Users/craga/Google Drive/TailDemography/outputFiles/Descriptive/'}}

### Choose Device

In [27]:
device = deviceDict['gandolf']
device

{'source': 'C:/Users/craga/Google Drive/TailDemography/outputFiles',
 'log': 'C:/Users/craga/Google Drive/TailDemography/Scripts and notes/Descriptive/',
 'output': 'C:/Users/craga/Google Drive/TailDemography/outputFiles/Descriptive/'}

# Source Data


### Logging

In [28]:
logging.basicConfig(filename=device['log']+'Desriptive Analyses.log'
                    , filemode='a',
                    format='%(funcName)s - %(levelname)s - %(message)s - %(asctime)s', level=logging.DEBUG)

## Functions

This section contains functions that were created for this notebook.

- [distribution](#distribution) #delete this we will use scipy stats describe instead
- [monthlit](#monthlit)
- [description](#description)
- [vocab_run](#vocab_run)

### distribution
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

*distribution* takes a series or list of numeric objects, *x*, and returns descriptive stats of x including
        n, minimum, maximum, median, sIQR, mean, and stdev
    
Here are a few examples of how *distribution* works.

In [29]:
foo = [0,1,2,'r']
distribution(foo)

In [30]:
bar = [0,1,2]
distribution(bar)

,n,minimum,maximum,median,siqr,mean,stdev
0,3,0,2,1.0,0.5,1.0,1.0


[Back to Functions](#Functions)

## monthlit
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

Here are a few examples of how _monthlit_ works.

In [31]:
dates = pd.DataFrame(data={'dates':['2018-12-9','2019-8-5', '2017/7/4',np.nan,None]})
dates.dates = pd.to_datetime(dates.dates)
dates

,dates
0,2018-12-09
1,2019-08-05
2,2017-07-04
3,NaT
4,NaT


In [32]:
np.isnan(np.nan)

True

In [33]:
monthlit(dates.dates.dt.month[0])

'Dec'

In [34]:
dates.dates.dt.month.apply(monthlit)

0    Dec
1    Aug
2    Jul
3    NaN
4    NaN
Name: dates, dtype: object

[Back to Functions](#Functions)

## description
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

In [35]:
def description(x,variable,percentage=False):
    if percentage:
            res = x[variable].describe()
            res[['mean','std','min','25%','50%','75%','max']] = res[['mean','std','min','25%','50%','75%','max']]\
            .apply(lambda x:x*100) 
#Need to Add CI calculation to this function
#             meanCI = 'not calculated'
    else:
        res = x[variable].describe() 
    res['siqr'] = (res['75%']-res['25%'])/2
    res['meanCI'] = 'not calculated'
    return res

### vocab_run
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

*vocab_run* takes a list, joins its the first the elements with a separator placing a different separator between
     the penultimate and final members of the list and returns the result as a string
     :param x: a list of strings to be concatenated
     :param connector_dict: a dictionary with keys describing the size of the list and values indicating the type of
     connectors separate the list elements.
    
Here are a few examples of how *vocab_run* works.

In [36]:
print("Could you bring some {} please?".format(vocab_run(['foo','bar','stuffkins'])))

Could you bring some foo, bar and stuffkins please?


In [37]:
print("You can either have {}.  You'll have to make a choice."\
      .format(vocab_run(['foo','bar','stuffkins'],connector_dict={1: None, 2: ' or ', 'run': ', '})))

You can either have foo, bar or stuffkins.  You'll have to make a choice.


[Back to Functions](#Functions)

We'll display all files in the source folder with the prefix _'cleaned CC data 2000-2017'_. The file names will be saved in a variable, _mysourcefiles_.

## Get Data
[Top](#Table-of-Contents)

Here we can set the locations from which we get data and to which we export it.

In [38]:
os.chdir(device['source'])
mysourcefiles = glob.glob('cleaned CC data 2000-2017_*.csv')
mysourcefiles

['cleaned CC data 2000-2017_2019-01-31 01hrs43min.csv']

In [39]:
pd.to_datetime(mysourcefiles[0].split("_")[1].split(".csv")[0].split(' ')[0])

Timestamp('2019-01-31 00:00:00')

Automate getting the latest file

In [40]:
[latestFile for latestFile in mysourcefiles if \
 max({pd.to_datetime(afile.split("_")[1].split(".csv")[0].split(' ')[0]) \
      for afile in mysourcefiles}) == pd.to_datetime(latestFile.split("_")\
                                                     [1].split(".csv")[0].split(' ')[0])]

['cleaned CC data 2000-2017_2019-01-31 01hrs43min.csv']

In [41]:
min({afile.split(' ')[-1].replace('hrs','').replace('min.csv','') for afile in mysourcefiles})

'0143'

In [42]:
latest = mysourcefiles[-1]
latest

'cleaned CC data 2000-2017_2019-01-31 01hrs43min.csv'

The most recent file is the one we will use as _df_ in our descriptive analysis.

In [50]:
df=pd.read_csv(latest)
origN = df.groupby('year').liznumber.nunique().reset_index()
df = df.loc[df.svl>=56]
newN = df.groupby('year').liznumber.nunique().reset_index()
df.date=pd.to_datetime(df.date)
df['month']=df.date.dt.month.apply(monthlit)
df.head()

Empty DataFrame
Columns: [year, liznumber]
Index: []


,species,toes_orig,sex,date,svl,tl,rtl,autotomized,mass,location,meters,newRecap,painted,sighting,paint.mark,vial,misc,rtl_orig,toes,toe_pattern,year,tl_svl,mass_svl,initialCaptureDate,year_diff,svl_diff,liznumber,sex_count,daysSinceCapture,capture,month
0,j,2-6-12-15,m,2010-06-19,80.0,110.0,29.0,True,20.0,10m v bottom bowl,-15,?,yes,NaN,y2c,03-10-cc,toe 15 missing at capture; possible recap,29.0,2-6-12-15,NaN,2010,1.375000,0.250000,2010-06-19,0,0.0,893,1,0,1,Jun
1,j,2-6-12-15,m,2010-10-07,86.0,102.0,28.0,True,17.0,6m v bottoom bowl right side 4 m up,NaN,R,yes,NaN,y22c,NaN,loose scale on ventrum and head,28.0,2-6-12-15,NaN,2010,1.186047,0.197674,2010-06-19,0,6.0,893,1,110,2,Oct
2,j,2-9-15-17,f,2010-08-13,56.0,77.0,0.0,False,5.5,20m up CCC,240,N,yes,NaN,y62c,61-10-cc,Tss,0.0,2-9-15-17,NaN,2010,1.375000,0.098214,2010-08-13,0,0.0,929,1,0,1,Aug
4,j,3-6-15-16,f,2010-08-18,72.0,62.0,46.0,True,11.0,halfway between pool and 2 falls 2m up rt side,385,N,yes,NaN,y65c,65-10-cc,NaN,46.0,3-6-15-16,NaN,2010,0.861111,0.152778,2010-08-18,0,0.0,947,1,0,1,Aug
5,j,3-6-12-17,m,2010-08-18,57.0,82.0,0.0,False,6.0,R outcrop ^ oak R,425,N,yes,NaN,y67c,66-10-cc,NaN,0.0,3-6-12-17,NaN,2010,1.438596,0.105263,2010-08-18,0,0.0,938,1,0,1,Aug


In [55]:
compTable = newN.rename(columns = {'liznumber':'adultLiz'})\
.merge(origN.rename(columns = {'liznumber':'allLiz'}), on=['year'])
compTable['comp'] = compTable.allLiz - compTable.adultLiz
compTable

,year,adultLiz,allLiz,comp
0,2000,124,155,31
1,2001,93,138,45
2,2002,115,221,106
3,2003,86,201,115
4,2004,69,123,54
5,2005,77,161,84
6,2006,8,11,3
7,2007,69,151,82
8,2008,75,126,51
9,2009,75,155,80


## Analyze Data
[Top](#Table-of-Contents)

We will first examine the range and distribution of number of variables in our data set:
- [Population Size](#Population-Size)
- [Species Distribution](#Species-Distribution)
- [Sex Distribution](#Sex-Distribution)
- [Tail Condition Distribution](#Tail-Condition-Distribution)
- [Severity of Autotomy](#Severity-of-Autotomy)
- [Location](#Location)
- [Morphometrics](#Morphometrics):
    - [SVL](#SVL)
    - [TL](#TL)
    - [RTL](#RTL)
    - [Mass](#Mass)
- [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
- [Captures](#Captures)
- [Growth](#Growth)
    - [SVL Growth](#SVL-Growth)
    - [TL Growth](#TL-Growth)
    - [RTL Growth](#RTL-Growth)
    - [NTL Growth](#NTL-Growth)
    - [Mass Growth](#Mass-Growth)
We will use first captures of each lizard in a year for these analyses.
- [Correlations to Population](#Correlations-to-Population)

## Reducing the analyses sample by date range and capture

In [44]:
df.loc[~(df.species=='j')].species.unique()

array(['v', 'other'], dtype=object)

In [45]:
df.loc[~(df.sex.isin(['m','f']))].liznumber.nunique()

7

In [46]:
sexdict= {'male':'m', 'f`':'f'}
df.loc[df.sex.isin(sexdict.keys()),'sex'] = df.loc[df.sex.isin(sexdict.keys()),'sex'].apply(lambda x: sexdict[x])
df.sex.unique()

array(['m', 'f', nan], dtype=object)

In [47]:
df.loc[~(df.sex.isin(['m','f']))].liznumber.nunique()

5

In [ ]:
monthsExcluded = ['Oct','Dec']
idx_exclusion = (df.month.isin(monthsExcluded))&(df.capture==1)&(df.species!='j')
print("The number of individuals captured for the first time in {} is {}. \
These are excluded for further analyses.".format(vocab_run(monthsExcluded,{1: None, 2: ' or ', 'run': ', '}),
                                                df.loc[idx_exclusion].liznumber.nunique()))
df=df.loc[~idx_exclusion]
df=df.loc[(df.species=='j')& (df.sex.isin(['m','f']))]

Here we create datasets including only the first or last sighting in each year for a given animal.

In [ ]:
df_lastInYear = df.loc[~df.duplicated(subset=['liznumber','year'],keep='last')]
df_firstInYear = df.loc[~df.duplicated(subset=['liznumber','year'])]

### Population Size

We will begin by examining the range and distribution of individuals in the population.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Add a column of counts of lizards weighted by the person-hours for the year, or provide argument based on asymptote analysis that we captured all lizards in the site.

In [ ]:
populationSize = df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()
populationSize['percChange'] = -(1-populationSize.liznumber/populationSize.liznumber.shift())
populationSize

 For some reason the first trace dissappears after I add the second one.  Need to fix this

In [ ]:
popSize = go.Scatter(x = populationSize.year
           , y = populationSize.liznumber
          ,name = 'population size')
percentChange = go.Scatter(x = populationSize.year
           , yaxis = 'y2', y = populationSize.percChange
          ,name = 'percent change')
data = [popSize]#,percentChange]
layout = go.Layout(
    title = 'Population Size for Sceloporus jarrovii 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18),
        rangemode = 'tozero'
        ),
    yaxis2 = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18),
        rangemode = 'tozero',
        side = 'right'
        )
)
fig = go.Figure(
        data = data,
        layout = layout)
#plot(fig, filename = 'Population Size for Sceloporus jarrovii.html')
iplot(fig, filename = 'Population Size for Sceloporus jarrovii.html')
# pio.to_image(fig, format='html')

We see a large decline in the <i>S. jarrovii</i> population size.

In [ ]:
description(populationSize,'liznumber')

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Sex Distribution

We will begin by examining the range and distribution of _sex_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Maybe aymptote analysis should be extended to examine asymptote by demographic factors as well.

In [ ]:
populationSize_sex = df_firstInYear.groupby(['year','sex']).liznumber.nunique().reset_index()\
.merge(df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()\
       .rename(columns={'liznumber':'liznumberYear'}),
       on=['year'])
populationSize_sex\
.loc[populationSize_sex.sex=='m','propMale'] = populationSize_sex\
.loc[populationSize_sex.sex=='m'].liznumber/populationSize_sex\
.loc[populationSize_sex.sex=='m'].liznumberYear
populationSize_sex\
.loc[populationSize_sex.sex=='f','propFemale'] = (populationSize_sex\
.loc[populationSize_sex.sex=='f'].liznumber/populationSize_sex\
.loc[populationSize_sex.sex=='f'].liznumberYear)
populationSize_sex.to_csv(device['output']+'population size 56 mm plus.csv',index = False)
populationSize_sex

In [ ]:
Sjarrovii = go.Scatter(x = populationSize_sex.loc[(populationSize_sex.propMale.notna())].year
           , y = populationSize_sex.loc[(populationSize_sex.propMale.notna())].propMale
          ,name = '<i>S. jarrovii</i>')
data = [Sjarrovii]
layout = go.Layout(
    title = 'Proportion of Males by Year for <i>Sceloporus jarrovii</i> 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        tickformat = ".2%",
        title = 'Percentage of Males',
        titlefont = dict(
            size = 18),
    range = [0,1])
    
)
fig = go.Figure(
        data = data,
        layout = layout)
# plot(fig, filename = 'Proportion of Males by Species and Year for Sceloporus jarrovii 2000-2017.html')
iplot(fig, filename = 'Proportion of Males by Species and Year for Sceloporus jarrovii 2000-2017.html')

In [ ]:
description(populationSize_sex,'propMale',True)[['mean','std','min','25%','50%','75%','max','siqr']]

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

To be Done

There is not much variation in the sex ratio in the population here, suggesting that these drops do not differentially  impact males or females overall.  We should confirm that there isn't a difference in particular age/size groups.  Such a difference might be explained by developmental or social changes which leave certain groups more vulnerable than others.

### Tail Condition Distribution

Here we look at the proportion of individuals in our data who have experienced autotomy.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [ ]:
populationSize_aut = df_firstInYear.groupby(['year','autotomized']).liznumber.nunique()\
.reset_index()\
.merge(df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()\
       .rename(columns={'liznumber':'liznumberYear'})
       ,on=['year'])
populationSize_aut\
.loc[populationSize_aut.autotomized,'propAutotomized'] = populationSize_aut\
.loc[populationSize_aut.autotomized].liznumber/populationSize_aut\
.loc[populationSize_aut.autotomized].liznumberYear
populationSize_aut\
.loc[~populationSize_aut.autotomized,'propIntact'] = (populationSize_aut\
.loc[~populationSize_aut.autotomized].liznumber/populationSize_aut\
.loc[~populationSize_aut.autotomized].liznumberYear)
populationSize_aut

In [ ]:
Sjarrovii = go.Scatter(x = populationSize_aut.loc[(populationSize_aut.propAutotomized.notna())].year
           , y = populationSize_aut.loc[(populationSize_aut.propAutotomized.notna())].propAutotomized
          ,name = 'Autotomized <i>S. jarrovii</i>')



data = [Sjarrovii]
layout = go.Layout(
    title = 'Proportion Autotomized by Year for <i>Sceloporus jarrovii</i> 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        tickformat=".2%",
        title = 'Proportion of Autotomized Lizards',
        titlefont = dict(
            size = 18),
    range=[0,1])
)
fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Proportion Autotomized by Year for Sceloporus jarrovii 2000-2017.html')
# plot(fig, filename = 'Proportion Autotomized by Year for Sceloporus jarrovii 2000-2017.html')

In [ ]:
description(populationSize_aut, 'propAutotomized',True)[['mean','std','min','25%','50%','75%','max','siqr']]

Proportion of _S. jarrovii_ with evidence of autotomy is also oscillates over the years.  
To be done:
- does this track with oscillations in the population?

### Severity of Autotomy
Here we will regress SVL and sex on TL for intact _S. jarrovii_ to estimate what the TL would have been had an animal not autotomized.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [ ]:
dfPred = df_firstInYear[['liznumber','svl','sex','tl','autotomized','year']]

In [ ]:
dfPred['tl_svl'] = dfPred.tl/dfPred.svl

In [ ]:
intactRatio = go.Box(y=dfPred.loc[dfPred.autotomized==False].tl_svl, name = 'Intact')
autotomizedRatio = go.Box(y=dfPred.loc[dfPred.autotomized].tl_svl, name = 'Autotmized')
data = [autotomizedRatio, intactRatio]
layout = go.Layout(
    title = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Autotomy Status',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL/SVL Ratio',
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)

iplot(fig, filename = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017.html')
# plot(fig, filename = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017.html')

We will identify the _Sceloporus jarrovii_ with a tl/svl ratio outside of the center 95%.  To do this, we create a boolean column _extremeRatio_ individuals with tl/svl ratios outside of the center 95% have a value of **True** in this column.  We only use individuals without extreme ratios to model tl and svl.

In [ ]:
print(dfPred.loc[dfPred.autotomized==False].tl_svl.describe())
dfPred.loc[(dfPred.autotomized==False)&
                   (dfPred.tl_svl>=dfPred.tl_svl.quantile(.025))&
                   (dfPred.tl_svl<=dfPred.tl_svl.quantile(.925)),'extremeRatio']= False
dfPred.loc[(dfPred.autotomized==False)&
                   (dfPred.tl_svl<dfPred.tl_svl.quantile(.025))|
                   (dfPred.tl_svl>dfPred.tl_svl.quantile(.925)),'extremeRatio']= True
dfPred.loc[dfPred.extremeRatio.isna()].autotomized.value_counts(dropna=False)


In [ ]:
sjM = pg.linear_regression(dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].svl,
                     dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].tl,
                     remove_na=True)
sjM

In [ ]:
sjF = pg.linear_regression(dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].svl,
                     dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].tl,
                     remove_na=True)
sjF

SVL and sex are sufficient to predict TL in males.

In [ ]:
femaleIntercept = [thing for thing in sjF.loc[sjF.names=='Intercept','coef']][0]
maleIntercept = [thing for thing in sjM.loc[sjM.names=='Intercept','coef']][0]
femaleSvl = [thing for thing in sjF.loc[sjF.names=='svl','coef']][0]
maleSvl = [thing for thing in sjM.loc[sjM.names=='svl','coef']][0]

print(femaleIntercept,maleIntercept,femaleSvl,maleSvl)

In [ ]:
dfPred.loc[dfPred.sex=='f','tlPred'] = dfPred.loc[dfPred.sex=='f'].svl*femaleSvl+femaleIntercept
dfPred.loc[dfPred.sex=='m','tlPred'] = dfPred.loc[dfPred.sex=='m'].svl*maleSvl+maleIntercept
dfPred['diff'] = dfPred.tlPred - dfPred.tl
dfPred['propDiff'] = dfPred['diff']/dfPred.tlPred

In [ ]:
dfPred.groupby(['autotomized','sex']).diff.describe()

In [ ]:
male = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].sort_values('svl')['tl'],
                  name = 'Male',mode='markers',marker=dict(size=5,opacity=0.8))
female = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].sort_values('svl')['tl'],
                    name = 'Female',mode='markers',marker=dict(size=5,opacity=0.8))
All = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)].sort_values('svl')['tl'],
                    name = 'All',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
maleOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='m')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='m')].sort_values('svl')['tl'],
                  name = 'Male Extreme',mode='markers',marker=dict(size=5,opacity=0.8))
femaleOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='f')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='f')].sort_values('svl')['tl'],
                    name = 'Female Extreme',mode='markers',marker=dict(size=5,opacity=0.8))
AllOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)].sort_values('svl')['tl'],
                    name = 'All Extreme',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))

data = [All,male,female,AllOut,maleOut,femaleOut]
layout = go.Layout(
    title = 'Scatter Plot of SVL vs TL for Intact <i>Sceloporus jarrovii</i> CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        tickangle=45,
        dtick = 1,
        title = 'SVL (mm)',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
#         tickformat = ".0%",
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)
# fig.update_xaxes(range=[0,dfPred.loc[(dfPred.autotomized==False)]['count'].describe()['max']+1])
# fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of SVL vs TL for Intact Sceloporus jarrovii CC 2000-2017.html')
# plot(fig, filename = 'Scatter Plot of SVL vs TL for Intact Sceloporus jarrovii CC 2000-2017.html')

In [ ]:
dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['autotomized','sex']).propDiff.describe()

In [ ]:
dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized'])\
.propDiff.describe()[['count','50%']].reset_index()

In [ ]:
dfYearlyEst = dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized','sex'])\
.propDiff.describe()[['count','50%']].reset_index(drop=False).merge(dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized'])\
.propDiff.describe()[['count','50%']].reset_index().rename(columns = {'count':'count_all',
                                                                      '50%':'50%_all'}),
                                                                    how = 'left',
                                                                    on =['year','autotomized'])
dfYearlyEst.loc[dfYearlyEst.autotomized]

In [ ]:
male = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')].sort_values('count_all')['count'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')].sort_values('count_all')['50%'],
                  name = 'Male',mode='markers')
female = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')].sort_values('count_all')['count'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')].sort_values('count_all')['50%'],
                  name = 'Female',mode='markers')
All = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)].sort_values('count_all')['count_all'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)].sort_values('count_all')['50%_all'],
                    name = 'All',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
medianAll = go.Scatter(x=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1],
                      y=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median(),
                        dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median()],
                         name='medianAll',mode='lines')
data = [All,male,female,medianAll]
layout = go.Layout(
    title = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Population Size',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
fig.update_xaxes(range=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1])
fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017.html')
# plot(fig, filename = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017.html')

In [ ]:
male = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['50%'],
                  name = 'Male',mode='markers')
female = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['50%'],
                  name = 'Female',mode='markers')
All = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'],
                    name = 'All',mode='lines',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
medianAll = go.Scatter(x=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'].describe()['min'],
                          dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'].describe()['max']],
                      y=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median(),
                        dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median()],
                         name='medianAll',mode='lines')
data = [All,male,female,medianAll]
layout = go.Layout(
    title = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
# fig.update_xaxes(range=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1])
# fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year.html')
# plot(fig, filename = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year.html')

To be done:

- for each sex and all, regress prop tl on relevant count 
- add these lines to plots with CI
- report on stat (what does this tell us?)

# resume
[TOC](#Table-of-Contents)

In [ ]:
male = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['50%'],
                  name = 'Male (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['count'].sum()))
female = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['50%'],
                    name = 'Female (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['count'].sum()))
All = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%'],
                    name = 'All (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count'].sum()))
data = [All,male,female,]
layout = go.Layout(
    title = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Population Size',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017.html')
# plot(fig, filename = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017.html')

In [ ]:
dfPred.loc[(dfPred.autotomized==True)].groupby('sex').propDiff.describe()

In [ ]:
female = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')]['year'], 
                y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')].propDiff
                      ,name='Females')
male = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')].propDiff
                    ,name='Males')
All = go.Box(x=dfPred.loc[(dfPred.autotomized==True)]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)].propDiff
                    ,name='All')
data = [All,male,female]
layout = go.Layout(
    title = 'Box Plot of Tail Lost at Capture for <i>Sceloporus jarrovii</i> by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'propDiff',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii by Year.html')
# plot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii by Year.html')

In [ ]:
female = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')]['year'], 
                y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')].propDiff
                      ,name='Females')
male = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')].propDiff
                    ,name='Males')
All = go.Box(x=dfPred.loc[(dfPred.autotomized==True)]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)].propDiff
                    ,name='All')
data = [All,male,female]
layout = go.Layout(
    title = 'Box Plot of Tail Lost at Capture for <i>Sceloporus jarrovii</i> by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'propDiff',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii by Year.html')
# plot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii by Year.html')

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Morphometrics

In this section we describe the distributions of various morphometrics.

- [SVL](#SVL)
- [TL](#TL)
- [RTL](#RTL)
- [Mass](#SVL)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

#### SVL

Now we examine the range and distribution of svl values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of SVL](#SVLhist)

In [ ]:
print("svl values in the data set range from {} to {} for and are distributed across sex \
as displayed here:"\
      .format(df_firstInYear.svl.min(), df_firstInYear.svl.max()))
description(df_firstInYear.groupby('sex'),variable='svl')

In [ ]:
SVLbyYear = description(df_firstInYear.groupby(['year','sex']),'svl').reset_index()
SVLbyYear

Let's plot these values. 

##### Figures of SVL values

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)


In [ ]:
female = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')].svl
                      ,name='Female Box',yaxis='y1')
male = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')].svl
                    ,name='Males Box',yaxis='y1')
female_line = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='f')]['year'], 
                y = SVLbyYear.loc[(SVLbyYear.sex=='f')]['50%']
                      ,name='Female Median', line = dict(color = 'red'),yaxis='y1')
male_line = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='m')]['year'], 
              y = SVLbyYear.loc[(SVLbyYear.sex=='m')]['50%']
                    ,name='Male Median', line = dict(color = 'blue')
                      ,yaxis='y1')
female_count = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='f')]['year'], 
                y = SVLbyYear.loc[(SVLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'),
                          yaxis = 'y1')
male_count = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='m')]['year'], 
              y = SVLbyYear.loc[(SVLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'),
                       yaxis = 'y1')

data = [male,female,male_line,female_line,male_count,female_count]
layout = go.Layout(
    title = 'Box Plot of SVL at Capture for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median SVL (mm)',
        titlefont = dict(
            size = 18)),
    yaxis2 = dict(
        title = 'Count of Lizard',
        titlefont = dict(
            size = 18),
        side = 'right'),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)
iplot(fig, filename = 'Box Plot of Median SVL Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'Box Plot of Median SVL Sceloporus jarrovii in CC 2000-2017.html')

To be done:
- look at correlation between the count and the median svl 
Outliers will be addressed in the Cleaning notebook, but will be removed for the remained of the analyses here.

#### TL

Now we examine the range and distribution of TL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of TL](#TLhist)

In [ ]:
TLbyYear = description(df_firstInYear.groupby(['year','autotomized','sex']),'tl').reset_index()
TLbyYear

##### Histogram of TL (Intact)

In [ ]:
femaleBoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')&~(df_firstInYear.autotomized)].tl
                      ,name='Females')
maleBoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')&~(df_firstInYear.autotomized)].tl
                    ,name='Males')
femaleIntact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                      y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['50%']
                      ,name='Intact Females')
maleIntact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['50%']
                    ,name='Intact Males')
female_count = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'))
male_count = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'))

data = [maleBoxIntact,femaleBoxIntact,maleIntact,femaleIntact,male_count,female_count]
layout = go.Layout(
    title = 'TL for Intact <i>Sceloporus jarrovii</i> in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
#iplot(fig, filename = 'TL for Intact Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'TL for Intact Sceloporus jarrovii in CC 2000-2017.html')

To be done:
- look at correlation between the count and the median tl 
Outliers will be addressed in the Cleaning notebook, but will be removed for the remained of the analyses here.

##### Histogram of TL (Autotomized)

In [ ]:
femaleBoxAutotomized = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')&(df_firstInYear.autotomized)].tl
                      ,name='Females')
maleBoxAutotomized = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')&(df_firstInYear.autotomized)].tl
                    ,name='Males')
femaleAutotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                      y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['50%']
                      ,name='Autotomized Females')
maleAutotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['50%']
                    ,name='Autotomized Males')
female_count = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'))
male_count = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'))

data = [maleBoxAutotomized,femaleBoxAutotomized,maleAutotomized,femaleAutotomized,male_count,female_count]
layout = go.Layout(
    title = 'TL for Autotomized <i>Sceloporus jarrovii</i> in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
#iplot(fig, filename = 'TL for Autotomized Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'TL for Autotomized Sceloporus jarrovii in CC 2000-2017.html')

##### Histogram of TL (Intact vs. Autotomized) 2

In [ ]:
TLbyYear.groupby('year')['year','50%'].apply(lambda x: x[['50%']])

In [ ]:
# BoxAut = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized)]['year'], 
#                 y = df_firstInYear.loc[(df_firstInYear.autotomized)].tl
#                       ,name='Autotomized')
# BoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized==False)]['year'], 
#               y = df_firstInYear.loc[(df_firstInYear.autotomized==False)].tl
#                     ,name='Intact')
compared = (TLbyYear.loc[~(TLbyYear.autotomized)]['50%'].reset_index(drop=True))\
-(TLbyYear.loc[(TLbyYear.autotomized)]['50%'].reset_index(drop=True))
Comparison = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)]['year'], 
                   y = compared,name='Intact - Autotomized',
                       mode = 'markers')
# Intact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)]['year'], 
#               y = TLbyYear.loc[~(TLbyYear.autotomized)]['50%']
#                     ,name='Intact')
data = [Comparison]
layout = go.Layout(
    title = 'Comparison on Median TL of <i>Sceloporus jarrovii</i> in CC 2000-2017 by Tail Condition',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
#iplot(fig, filename = 'Comparison on Median TL of Sceloporus jarrovii in CC 2000-2017.html')
plot(fig, filename = 'Comparison on Median TL of Sceloporus jarrovii in CC 2000-2017.html')

##### Histogram of TL (Intact vs. Autotomized)

In [ ]:
BoxAut = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized)]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.autotomized)].tl
                      ,name='Autotomized')
BoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized==False)]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.autotomized==False)].tl
                    ,name='Intact')
# Autotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)]['year'], 
#                    y = TLbyYear.loc[(TLbyYear.autotomized)]['50%']
#                       ,name='Autotomized')
# Intact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)]['year'], 
#               y = TLbyYear.loc[~(TLbyYear.autotomized)]['50%']
#                     ,name='Intact')
data = [BoxAut,BoxIntact]
layout = go.Layout(
    title = 'TL of <i>Sceloporus jarrovii</i> in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'TL of Sceloporus jarrovii in CC 2000-2017.html')
#plot(fig, filename = 'TL of Sceloporus jarrovii in CC 2000-2017.html')

#### RTL

Now we examine the range and distribution of RTL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of RTL](#RTLhist)

In [ ]:
RTLbyYear = description(df_firstInYear.loc[df_firstInYear.autotomized].groupby(['year','sex']),'rtl').reset_index()
RTLbyYear

##### Histogram of rtl

In [ ]:
femaleAut = go.Scatter(x=RTLbyYear.loc[(RTLbyYear.sex=='f')]['year'], 
                   y = RTLbyYear.loc[(RTLbyYear.sex=='f')]['50%']
                      ,name='Females')
maleAut = go.Scatter(x=RTLbyYear.loc[(RTLbyYear.sex=='m')]['year'], 
              y = RTLbyYear.loc[(RTLbyYear.sex=='m')]['50%']
                    ,name='Males')

data = [maleAut,femaleAut]
layout = go.Layout(
    title = 'Median RTL for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median RTL (mm)',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median RTL Sceloporus jarrovii in CC 2000-2017.html')
#plot(fig, filename = 'Median RTL Sceloporus jarrovii in CC 2000-2017.html')

#### Mass

Now we examine the range and distribution of mass values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Plot of mass](#Plot-of-mass)

We probably need to adjust this analysis to consider teh month in which the females were captures since gravidity or nearness to the time of parturition may bias the analysis.

In [ ]:
MassbyYear = description(df_firstInYear.loc[df_firstInYear.autotomized].groupby(['year','sex']),'mass').reset_index()
MassbyYear

##### Plot of mass

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [ ]:
female = go.Scatter(x=MassbyYear.loc[(MassbyYear.sex=='f')]['year'], 
                   y = MassbyYear.loc[(MassbyYear.sex=='f')]['50%']
                      ,name='Females')
male = go.Scatter(x=MassbyYear.loc[(MassbyYear.sex=='m')]['year'], 
              y = MassbyYear.loc[(MassbyYear.sex=='m')]['50%']
                    ,name='Males')

data = [male,female]
layout = go.Layout(
    title = 'Median Mass for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Mass (mm)',
        titlefont = dict(
            size = 18),
    range = [0,MassbyYear['50%'].max()+5]),
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median Mass Sceloporus jarrovii in CC 2000-2017.html')
#plot(fig, filename = 'Median Mass Sceloporus jarrovii in CC 2000-2017.html')

## Captures

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.  We will use all captures for this

In [ ]:
df_firstInYear.groupby('liznumber').capture.apply(lambda x: x.values)

In [ ]:
print("The maximum number of captures among lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df_firstInYear.groupby('liznumber').capture.max().min(),
              df_firstInYear.groupby('liznumber').capture.max().max()))
captureMedYr = description(df_firstInYear.groupby(['year','sex']),variable='capture')[['50%','75%','max']].reset_index()
captureMedYr

In [ ]:
female = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='f')]['year'], 
                   y = captureMedYr.loc[(captureMedYr.sex=='f')]['50%']
                      ,name='Females')
male = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='m')]['year'], 
              y = captureMedYr.loc[(captureMedYr.sex=='m')]['50%']
                    ,name='Males')
# femaleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='f']['year'], 
#                    y = df_firstInYear.loc[df_firstInYear.sex=='f'].groupby('liznumber').capture.apply(lambda x: x.values)
#                       ,name='Females')
# maleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='m']['year'], 
#               y = df_firstInYear.loc[df_firstInYear.sex=='m'].groupby('liznumber').capture.apply(lambda x: x.values)
#                     ,name='Males')

data = [male,female]#,maleBox,femaleBox]
layout = go.Layout(
    title = 'Median Years Captured for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Years Captured',
        titlefont = dict(
            size = 18),
    range = [0,captureMedYr['50%'].max()+1]),
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median Years Captured Sceloporus jarrovii in CC 2000-2017.html')
#plot(fig, filename = 'Median Years Captured  Sceloporus jarrovii in CC 2000-2017.html')

In [ ]:
# female = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='f')]['year'], 
#                    y = captureMedYr.loc[(captureMedYr.sex=='f')]['50%']
#                       ,name='Females')
# male = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='m')]['year'], 
#               y = captureMedYr.loc[(captureMedYr.sex=='m')]['50%']
#                     ,name='Males')
femaleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='f']['year'], 
                   y = df_firstInYear.loc[df_firstInYear.sex=='f'].groupby('liznumber').capture.apply(lambda x: x.values)
                      ,name='Females')
maleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='m']['year'], 
              y = df_firstInYear.loc[df_firstInYear.sex=='m'].groupby('liznumber').capture.apply(lambda x: x.values)
                    ,name='Males')

data = [maleBox,femaleBox]
layout = go.Layout(
    title = 'Median Years Captured for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Years Captured',
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Years Captured Sceloporus jarrovii in CC 2000-2017.html')
#plot(fig, filename = 'Box Years Captured  Sceloporus jarrovii in CC 2000-2017.html')

In order to interpret these data we need to factor in person-hours for each year.

## Years

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of years over which that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of years over which each lizard was captures and finally count the number of lizards that have a given number of years over which they were captured.  We will use all captures for this.

In [ ]:
(df_firstInYear.groupby('liznumber').year_diff.max()+1).value_counts(normalize=True).reset_index()\
.rename(columns={'index':'year_diff','year_diff':'proportion'}).sort_values('year_diff')

In [ ]:
print("The number of years over which each lizard was captured among lizards in the data set range from {} to {}\
for and are distributed across species and sex as displayed here:"\
      .format(df_firstInYear.groupby('liznumber').year_diff.max().min()+1, 
              df_firstInYear.groupby('liznumber').year_diff.max().max()+1))
description(df_firstInYear.groupby('sex'),variable='year_diff')

In [ ]:
Males = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex=='m')].groupby('liznumber').capture.max(),
                     name='Males')
Females = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex=='f')].groupby('liznumber').capture.max(),
                       name='Females')

data = [Males,Females]
layout = go.Layout(
    title = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Unique Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017.html')
#plot(fig, filename = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017.html')

### Maximum Number of Captures based on Tail condition
 - match for species, sex, size and location

In [ ]:
intactFemale = go.Histogram(x = df_firstInYear.loc[(~df_firstInYear.autotomized)&
                                                   (df_firstInYear.sex=='f')].groupby('liznumber').capture.max()
                      ,name='intact females')
intactMale = go.Histogram(x = df_firstInYear.loc[(~df_firstInYear.autotomized)&
                                                 (df_firstInYear.sex=='m')].groupby('liznumber').capture.max()
                      ,name='intact males')
autotomizedFemale = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.autotomized)&
                                                        (df_firstInYear.sex=='f')].groupby('liznumber').capture.max()
                           ,name='autotomized females')
autotomizedMale = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.autotomized)&
                                                      (df_firstInYear.sex=='m')].groupby('liznumber').capture.max()
                           ,name='autotomized males')

data = [intactFemale,intactMale,autotomizedFemale,autotomizedMale]
layout = go.Layout(
    title = 'Maximum Number of Captures by Tail Condition 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Captures by Tail Condition in Crystal Creek 2000 - 2017')

## Growth

- [SVL Growth](#SVL-Growth)
- [TL Growth](#TL-Growth)
- [RTL Growth](#RTL-Growth)
- [Mass Growth](#Mass-Growth)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.

### SVL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate?

In [ ]:
df_firstInYear['svl_growth'] = df_firstInYear.loc[df_firstInYear.year_diff>0]\
.apply(lambda x: x.svl_diff/x.year_diff, axis=1)
df_firstInYear.loc[df_firstInYear.svl_growth.notna()]
                                                                         

In [ ]:
print("svl_growth values in the data set range from {} to {} for and are distributed across sex \
as displayed here:"\
      .format(df_firstInYear.svl_growth.min(), df_firstInYear.svl_growth.max()))
description(df_firstInYear.groupby('sex'),variable='svl_growth')

In [ ]:
svl_growthbyYear = description(df_firstInYear.groupby(['year','sex']),'svl_growth').reset_index()
svl_growthbyYear

Let's plot these values. 

##### Figures of SVL values

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)


In [ ]:
female = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')].svl_growth
                      ,name='Female Box',yaxis='y1')
male = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')].svl_growth
                    ,name='Males Box',yaxis='y1')
female_line = go.Scatter(x=svl_growthbyYear.loc[(svl_growthbyYear.sex=='f')]['year'], 
                y = svl_growthbyYear.loc[(svl_growthbyYear.sex=='f')]['50%']
                      ,name='Female Median', line = dict(color = 'red'),yaxis='y1')
male_line = go.Scatter(x=svl_growthbyYear.loc[(svl_growthbyYear.sex=='m')]['year'], 
              y = svl_growthbyYear.loc[(svl_growthbyYear.sex=='m')]['50%']
                    ,name='Male Median', line = dict(color = 'blue')
                      ,yaxis='y1')
female_count = go.Scatter(x=svl_growthbyYear.loc[(svl_growthbyYear.sex=='f')]['year'], 
                y = svl_growthbyYear.loc[(svl_growthbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'),
                          yaxis = 'y1')
male_count = go.Scatter(x=svl_growthbyYear.loc[(svl_growthbyYear.sex=='m')]['year'], 
              y = svl_growthbyYear.loc[(svl_growthbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'),
                       yaxis = 'y1')

data = [male,female,male_line,female_line]
layout = go.Layout(
    title = 'Box Plot of SVL Growth for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median SVL Growth (mm)',
        titlefont = dict(
            size = 18)),
    yaxis2 = dict(
        title = 'Count of Lizard',
        titlefont = dict(
            size = 18),
        side = 'right'),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)
iplot(fig, filename = 'Box Plot of Median SVL Growth Sceloporus jarrovii in CC 2000-2017.html')
# plot(fig, filename = 'Box Plot of Median SVL Growth Sceloporus jarrovii in CC 2000-2017.html')

In [ ]:
popVar = 'liznumberYear'
summerPred = df_reg_season.loc[(df_reg_season.source=='portal')&
                                               (df_reg_season.season.isin(['summer']))]\
                             [['popinYearplus1','TMIN 50%']].dropna()

sjWsumm = pg.linear_regression(summerPred[['TMIN 50%']],
                     summerPred['popinYearplus1'],
                     remove_na=True)

sjWsumm

turn this into a function and then groupby and apply

df['svl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['svl'].max()-x['svl'].min())/(x['year_diff'].max()+1))

### TL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the tail size growth rate?

df['tl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['tl'].max()-x['tl'].min())/(x['year_diff'].max()+1))

### RTL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the regrown tail size growth rate?

df['rtl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['rtl'].max()-x['rtl'].min())/(x['year_diff'].max()+1))

### NTL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Mass Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate in terms of MAss?

df['mass_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['mass'].max()-x['mass'].min())/(x['year_diff'].max()+1))

tmp = df.groupby('liznumber').mass_growth_ann.nunique().reset_index()
check = tmp.loc[tmp.mass_growth_ann>1,'liznumber']
print(len(check))
check

## Correlations to Population

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [ ]:
def candidate(m,dv,placement=(1,1)):
    assert(dv in m.columns)
    return m[dv].sort_values().reset_index().iloc[placement[0]:placement[1]+1,:]

from functools import reduce

def topcorr(corrdf,lowestrank,dvs):
    candidates = [candidate(corrdf,dv,(1,lowestrank)) for dv in dvs]
    merger =  reduce(lambda x, y: pd.merge(x, y, on = 'index', how = 'outer'), candidates).fillna('--')
    return merger

I need to create a df with with values to be correlated:
- count
- prop Male
- SVL
- TL
- RTL
- Mass
- TL Severity
- Sex Distr


In [ ]:
dfCorrPrep = populationSize_sex.loc[populationSize_sex.sex=='f',
                                    ['year', 'sex', 'liznumber', 
                                     'liznumberYear','propFemale']]\
.merge(populationSize_aut.loc[populationSize_aut.propAutotomized.notna(),
                              ['year','liznumberYear', 'propAutotomized']], 
                         how = 'left', on = ['year','liznumberYear'])\
.merge(dfYearlyEst.loc[dfYearlyEst.autotomized,['year', 'sex', 
                                                'count_all', '50%_all']],
       on = ['year','sex'])\
.rename(columns = {'count_all':'nAutotomized', 
                   '50%_all':'medianPropTailAutotomized',
                                             'liznumber':'numberFemale'})
dfCorrPrep = dfCorrPrep[['liznumberYear', 
                         'propFemale', 'propAutotomized', 
                         'medianPropTailAutotomized']]
dfCorrPrep

In [ ]:
#Dropping proportion of Females, but will put it back once I can order the y-axis
dfCorr = dfCorrPrep.corr()
dfCorr = topcorr(dfCorr,2,dfCorr.columns.tolist()) 
dfCorr = dfCorr.set_index('index')
dfCorr

In [ ]:
testx = dfCorr.columns
testy = dfCorr.index
testz = dfCorr.values
test = go.Figure(go.Heatmap(x=testx,y=testy,z=testz))
#plot(test, filename = 'population correlation matrix.html')
iplot(test, filename = 'population correlation matrix.html')

## Export Files
[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)